## Дизайн AB-теста

Проведение экспериментов — одна из основных и регулярных задач продуктового аналитика. Этот ноутбук рассматривает процесс дизайна и сложности, с которыми можно столкнуться, решая задачу (с фокусом на метрики).

После того, как сформулирована гипотеза, аналитик должен ответить на следующие вопросы:
1. Какая единица рандомизации эксперимента?
2. На какой сегмент будет запущен эксперимент?
3. Какой нужен размер выборки?
4. Как долго будет идти эксперимент?

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import norm

import warnings
warnings.filterwarnings('ignore')

### 1. Единица рандомизации эксперимента

Классические AB-тесты принято запускать с поюзерным разбиением. Однако, такой формат тестирования подходит не для всех гипотез. К другим распространенным подходам относят:
- Свитчбек-тесты — единица рандомизации время или время-пространство;
- Региональные (гео) тесты — единица рандомизации город или регион.

Мы переходим к другим единицам рандомизации, когда проведение поюзерных тестов может исказить результаты из-за сетевого эффекта.

### 2. Сегмент, который попадет в тест

Определяется:
- гипотезой (если она направлена на конкретную группу пользователей);
- местом фичи в продукте (где именно возникает взаимодействие).

Разбиение пользователей можно начинать в момент, когда они сталкиваются с экспериментируемым элементом. Например, если мы тестируем пошаговое оформление покупки вместо единой формы, в эксперимент попадут только пользователи, начавшие процесс оформления.

### 3. Размер выборки

Способ определения размера выборки зависит от критерия, который будем применять к оценке результатов.    
Наиболее распространенный подход — расчёт через MDE. При этом в формуле используются квантильные значения стандартного нормального распределения (z-критерий).

Для определения размера выборки через MDE нужно задать:
- размер эффекта, который мы хотим задетектировать (MDE);
- уровни ошибок I и II рода;
- дисперсию метрики (базовый показатель конверсии для конверсий).

**Для среднего**:

$$
n \;=\; \frac{2\,\sigma^2 \,\big(z_{1-\alpha/2}+z_{1-\beta}\big)^2}{\text{MDE}^2}
$$


**Для доли** (бинарная метрика):

$$
n \;\\=\;\; \frac{2\,p(1-p)\,\big(z_{1-\alpha/2}+z_{1-\beta}\big)^2}{\text{MDE}^2}
$$

- Формулы указаны для групп одинакового размера (50/50) и двустороннего критерия;
- n — размер одной группы.

### 4. Длительность эксперимента

Зависит от размера выборки. Правильный подход — на пред периоде симулировать накопление уникальных пользователей, взаимодействоваших с функционалом, и вернуть число дней, за которое собрался нужный объем. 

Так как мы рассматриваем дизайн с точки зрения используемых метрик, давайте просто возьмем среднее число уникальных пользователей в день по месячным данным. Но важно помнить, что такая оценка очень грубая и может давать заметную погрешность.

### Case Study — дизайн A/B-теста для мобильной игры


**Легенда**   
Представим, что мы аналитики в команде мобильной игры. К нам пришел продакт и просит провести дизайн эксперимента — мы усложняем уровни игры, начиная с 10 уровня, т.к. хотим вырастить средний чек (пользователям будет сложнее пройти дальше, поэтому они будут покупать более дорогие и эффективные атрибуты).

**Гипотеза** — усложнение прохождения игры, начиная с 10 уровня, увеличит средний чек покупки как минимум на 2.5%.

1. Единица рандомизации — поюзерный
2. Сегмент, который попадет в тест — 10+ уровень
3. Размер выборки —

In [23]:
df = pd.read_csv('data/seminar_3_mobile_game.csv')

df.head()

,date,user_id,level,event_id,purchase_amount
0,2025-08-01,1,4,8,NaN
1,2025-08-01,1,4,13,NaN
2,2025-08-01,1,4,12,NaN
3,2025-08-01,1,4,8,NaN
4,2025-08-01,2,1,3,NaN


In [24]:
df.shape

(2168137, 5)

In [25]:
df.user_id.nunique()

20000

In [26]:
df_10_lvl = df[df['level'] >= 10].reset_index(drop=True)

Покупка — event_id = 10

In [27]:
df_10_lvl[df_10_lvl['event_id'] == 10].purchase_amount.mean()

25.872720063200756

In [28]:
def sample_size_mean(metric_series, mde, alpha=0.05, power=0.8, two_sided=True):
    
    var = np.var(metric_series, ddof=1)
    
    if two_sided:
        z_alpha = norm.ppf(1 - alpha/2)
    else:
        z_alpha = norm.ppf(1 - alpha)
    z_beta = norm.ppf(power)
    
    n = (2 * var * (z_alpha + z_beta) ** 2) / (mde ** 2)
    
    return int(np.ceil(n))

In [29]:
sample_size_mean(df_10_lvl[df_10_lvl['event_id'] == 10].purchase_amount, 
                 df_10_lvl[df_10_lvl['event_id'] == 10].purchase_amount.mean() * 0.025, 
                 alpha=0.05, power=0.8, two_sided=True)

8860

Не кажется ли вам, что в дизайне есть что-то подозрительное?

Средний чек — ratio-метрика. Единица расчета (покупка) в ней отличается от единицы рандомизации эксперимента (пользователя). 

In [30]:
df_10_lvl['is_purchase'] = df_10_lvl['event_id'] == 10

In [31]:
df_users_ratio = df_10_lvl.groupby(['user_id'], as_index=False) \
    .agg(sum_amt=('purchase_amount', 'sum'),
         purchases=('is_purchase', 'sum'))

In [32]:
df_users_ratio['avg_check_per_user'] = (df_users_ratio['sum_amt'] / df_users_ratio['purchases']).fillna(0)

In [33]:
df_users_ratio.isna().sum()

user_id               0
sum_amt               0
purchases             0
avg_check_per_user    0
dtype: int64

In [34]:
sample_size_mean(df_users_ratio.avg_check_per_user, 
                 df_users_ratio.avg_check_per_user.mean() * 0.025, 
                 alpha=0.05, power=0.8, two_sided=True)

12349

In [35]:
df_users_ratio.avg_check_per_user.mean()

20.410371413919382

In [36]:
df_10_lvl[df_10_lvl['event_id'] == 10].purchase_amount.mean()

25.872720063200756

Получилась другая метрика

Если подумать, у нас возникает зависимость между числителем и знаменателем — если покупка не будет сделана, значение чека в числителе просто не появится!

Что же делать?

### Ratio-метрики

В A/B-тестах без ratio-метрик не обойтись. Благодаря формуле они часто оказываются наиболее удачными:
- устойчивы;
- интерпретируемы.

Поэтому в аналитике разработаны специальные подходы для корректной работы с ratio-метриками.

**1. Переход к поюзерной метрике.**

Перевести ratio на уровень пользователя, чтобы единица анализа совпадала с единицей рандомизации. Например, вместо среднего чека брать ARPU. 

Плюсы: независимость наблюдений, корректный тест.     
Минусы: это совсем другая метрика, направление которой может не совпадать с изначальной. Если какая-то часть пользователей просто начнет покупать чаще, то мы увидим рост ARPU без роста среднего чека. В таком случае решение по эксперименту может быть ошибочным.

**2. Дельта-метод**

Использует формулу для оценки дисперсиии отношения двух случайных величин:

$$
\mathrm{Var}\!\left(\frac{X}{Y}\right) \;\approx\; 
\frac{1}{\mu_Y^2}\,\mathrm{Var}(X) 
\;+\; \frac{\mu_X^2}{\mu_Y^4}\,\mathrm{Var}(Y) 
\;-\; 2\,\frac{\mu_X}{\mu_Y^3}\,\mathrm{Cov}(X,Y)
$$

Плюсы: быстрый метод, который хорошо подходит для больших выборок.   
Минусы: может быть не точным при небольших n.

In [18]:
def sample_size_ratio_delta(num, den, mde, alpha=0.05, power=0.8, two_sided=True):
    mu_X = num.mean()
    mu_Y = den.mean()
    
    var_X = np.var(num, ddof=1)
    var_Y = np.var(den, ddof=1)
    cov_XY = np.cov(num, den, ddof=1)[0,1]

    var_ratio = (var_X / mu_Y**2
                 - 2 * mu_X * cov_XY / mu_Y**3
                 + (mu_X**2) * var_Y / mu_Y**4)

    if two_sided:
        z_alpha = norm.ppf(1 - alpha/2)
    else:
        z_alpha = norm.ppf(1 - alpha)
    z_beta = norm.ppf(power)

    n = (2 * var_ratio * (z_alpha + z_beta)**2) / (mde**2)
    return int(np.ceil(n))

In [37]:
sample_size_ratio_delta(df_users_ratio.sum_amt, df_users_ratio.purchases, 
                        (df_users_ratio['sum_amt'].sum() / df_users_ratio['purchases'].sum()) * 0.025,  
                        alpha=0.05, power=0.8, two_sided=True)

3699

In [38]:
unique_users_per_day = df.user_id.nunique() / df['date'].nunique()

In [39]:
n = sample_size_ratio_delta(df_users_ratio.sum_amt, df_users_ratio.purchases, 
                        (df_users_ratio['sum_amt'].sum() / df_users_ratio['purchases'].sum()) * 0.025,  
                        alpha=0.05, power=0.8, two_sided=True)

In [40]:
n * 2 /unique_users_per_day

11.4669